In [4]:
# Description
#
#
#

import wrds
import feather

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [5]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection(wrds_username='amglex')

Loading library list...
Done


In [130]:
######################################
# Step 1                             #
# CRSP Block                         #
######################################

# set sample date range
begdate = '03/01/2008'
enddate = '12/31/2008'

In [11]:
# sql similar to crspmerge macro

crsp_m = conn.raw_sql(f"""
                      select permno, date, cfacpr, cfacshr, ret, shrout, prc
                      from crsp.msf as a
                      where a.date between '{begdate}' and '{enddate}'
                      """, date_cols=['date']) 

In [79]:
check = crsp_m
crsp_m.head()

,permno,date,cfacpr,cfacshr,ret,shrout,prc
0,10001.0,2008-03-31,1.0,1.0,-0.052516,4347.0,8.965
1,10001.0,2008-04-30,1.0,1.0,-0.036698,4347.0,8.600
2,10001.0,2008-05-31,1.0,1.0,0.271628,4347.0,10.900
3,10001.0,2008-06-30,1.0,1.0,-0.010092,4349.0,10.750
4,10001.0,2008-07-31,1.0,1.0,-0.053023,4349.0,10.140


In [80]:
crsp_m.date = pd.DatetimeIndex(crsp_m.date)

In [83]:
# Change all dates to last day of respective month for later joining

import datetime

def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)

crsp_m.date = crsp_m.date.apply(last_day_of_month)

# Test to check if all dates are month end -> correct if = 0
date = pd.DatetimeIndex(crsp_m.date)
sum(date.is_month_end == 0)
del(date)

In [92]:
# Calculate adjusted SHROUT and Price

crsp_m['TSO'] = crsp_m.shrout * crsp_m.cfacshr * 1000 # Compute "Adjusted total shares outstanding"
crsp_m['P'] = abs(crsp_m.prc)/crsp_m.cfacpr           # "Price at t period end, adjusted"
crsp_m['P_1'] = crsp_m.P.shift(1)                     # "Price at t-1, adjusted"

In [97]:
crsp_m = crsp_m.drop(columns = ['prc', 'cfacpr', 'shrout'])

In [ ]:
/* Step 1.3. Get report and vintage dates from Thomson-Reuters Mutual Fund Holdings */
/* Exclude Non-Equity Funds from Holdings data that is reported as of Fiscal Quarter End */
/* First, Keep First Vintage with Holdings Data for Each RDATE-FUNDNO */
proc sql;
create table First_Vint
as select distinct intnx("month",rdate,0,"E") as rdate format date9., fdate, fundno
from tfn.s12type1
where ("&begdate"d <= rdate <="&enddate"d and ioc not in (1,5,6,7))
group by fundno, intnx("month",rdate,0,"E")
having fdate=min(fdate) and max(rdate)=rdate
order by fundno, rdate desc;
quit;

In [ ]:
# Step 1.3. Get report and vintage dates from Thomson-Reuters Mutual Fund Holdings
# Exclude Non-Equity Funds from Holdings data that is reported as of Fiscal Quarter End
# First, Keep First Vintage with Holdings Data for Each RDATE-FUNDNO

In [133]:
TR = conn.raw_sql(f"""
                    SELECT distinct rdate, fdate, fundno
                    FROM tfn.s12type1
                    WHERE rdate between '{begdate}' and '{enddate}'
                    AND ioc not in (1,5,6,7)
                    """, date_cols=['rdate','fdate']) 

TR.dtypes

rdate     datetime64[ns]
fdate     datetime64[ns]
fundno           float64
dtype: object

In [135]:
TR.rdate = TR.rdate.apply(last_day_of_month)

# Test to check if all dates are month end -> correct if = 0
date = pd.DatetimeIndex(TR.rdate)
print(sum(date.is_month_end == 0))
del(date)

0


In [144]:
TR.groupby(by = ['fundno', 'rdate']).count()

fdate
fundno  rdate            
10.0    2008-03-31      1
        2008-06-30      2
        2008-09-30      1
12.0    2008-03-31      1
        2008-06-30      1
        2008-09-30      1
        2008-12-31      1
15.0    2008-03-31      1
        2008-06-30      1
        2008-09-30      1
        2008-12-31      1
25.0    2008-03-31      1
        2008-06-30      1
        2008-09-30      1
        2008-12-31      1
27.0    2008-03-31      1
        2008-06-30      1
        2008-09-30      1
        2008-12-31      1
28.0    2008-03-31      2
        2008-09-30      1
        2008-12-31      2
29.0    2008-06-30      2
        2008-12-31      2
39.0    2008-03-31      1
        2008-06-30      1
        2008-09-30      1
        2008-12-31      1
40.0    2008-03-31      2
        2008-09-30      1
...                   ...
96828.0 2008-09-30      1
96830.0 2008-06-30      3
96833.0 2008-07-31      2
96834.0 2008-05-31      4
        2008-11-30      1
96836.0 2008-07-31      2
96837.0 2008-07-31      3
96838.0 2008-06-30      2
        2008-12-31      2
96839.0 2008-06-30      2
        2008-12-31      2
96840.0 2008-04-30      3
        2008-10-31      1
96841.0 2008-03-31      4
96853.0 2008-07-31      2
96857.0 2008-06-30      3
        2008-12-31      1
96862.0 2008-06-30      2
        2008-12-31      1
96864.0 2008-06-30      1
        2008-12-31      2
96867.0 2008-06-30      2
        2008-09-30      1
        2008-12-31      1
96869.0 2008-08-31      1
96870.0 2008-06-30      3
        2008-12-31      1
96877.0 2008-08-31      2
96879.0 2008-08-31      2
96903.0 2008-04-30      5

[7409 rows x 1 columns]

In [2]:
# Connect to DB
db = wrds.Connection(wrds_username='amglex')
print('Successfully connected')

/Users/alexandersteeb/.virtualenvs/mf/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Loading library list...
Done
Successfully connected


In [3]:
tfn = "/wrds/tfn/sasdata/s12";
mfl = "/wrds/mfl/sasdata";
ff = "/wrds/ff/sasdata";
 
# Step 1.1. Specifying Options
begdate = '1980-03-01'
enddate = '2008-12-31'

In [ ]:
######################
# Query the data
######################

print('Start downloading data ...')

# SQL Query
data_raw_df = db.raw_sql(
    '''
    SELECT crsp_company_key, security_name
    FROM holdings 
    LIMIT 10000000   
    '''
)

print('SQL successful')

print(data_raw_df.shape)

print(data_raw_df.dtypes)

print(data_raw_df.head())

path = '../../data/raw/holdings_co_info.feather'

feather.write_dataframe(data_raw_df, path)

print("Successfully saved data")